## Load data

In [1]:
import pickle 
import pandas as pd
import numpy as np
import feather
from tqdm import tqdm
from sklearn.cross_validation import train_test_split
# Load latent factors
orig_latent_factors = pd.read_csv('./data/song_latent_factors.csv')

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#audio_features = pd.read_pickle('./data/processedAudioFeatures/audio_features_part41.pkl')
#for i in tqdm(range(42)):
#    print(i)
#    dataPath = './data/processedAudioFeatures/audio_features_part'+str(i)+'.pkl'
#    print(dataPath)
#    audio_features = pd.read_pickle(dataPath)
#    feather.write_dataframe(audio_features, './data/processedAudioFeatures/audio_features_part'+str(i)+'.feather')
    

In [3]:
#namesList = []
#for i in tqdm(range(42)):
#    print(i)
#    dataPath = './data/processedAudioFeatures/audio_features_part'+str(i)+'.feather'
#    print(dataPath)
#    audioSmallDF = feather.read_dataframe(dataPath)
#    namesList.append(audioSmallDF.columns)

#dfAudioFeaturesNameList = pd.DataFrame(namesList)
#feather.write_dataframe(dfAudioFeaturesNameList, '/Users/dmarcous/git/wavenetmusicrecommendation/data/mels_songs_list.feather')



In [4]:
bad_songs = orig_latent_factors.columns[orig_latent_factors.isnull().any()]

In [5]:
song_train_test_split = feather.read_dataframe('./data/songs_train_test.feather')
clean_song_split = song_train_test_split[[x not in bad_songs for x in song_train_test_split.song]]
print(song_train_test_split.shape)
print(clean_song_split.shape)
#song_train_test_split

(72721, 2)
(72721, 2)


In [6]:
train_songs = song_train_test_split.loc[clean_song_split.loc[:,'label']=='train',['song']]['song'].tolist()
test_songs = song_train_test_split.loc[clean_song_split.loc[:,'label']=='test',['song']]['song'].tolist()

In [7]:
splitted_train_songs, splitted_valid_songs = train_test_split(train_songs)

In [8]:
clean_latent_factors = orig_latent_factors[clean_song_split['song']]
clean_song_list = clean_song_split['song'].tolist()

In [9]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(clean_latent_factors)
latent_factors = pd.DataFrame(scaler.transform(clean_latent_factors),columns=clean_latent_factors.columns)
#latent_factors

In [10]:
from os import listdir
from os.path import isfile, join
embeddings_path = '/Users/dmarcous/git/wavenetmusicrecommendation/data/wavenetEmbeddings'
embeddings_list = [f for f in listdir(embeddings_path) if isfile(join(embeddings_path, f))]
embedding_songs_list_full = [(song.strip('_embeddings.npy'), embeddings_path+'/'+song) for song in embeddings_list]

In [11]:
embedding_songs_list = [x for x in embedding_songs_list_full if x[0] in clean_song_list]
print(len(embedding_songs_list))
#print((embedding_songs_list[1:5]))
#print(clean_song_list[1:5].tolist())
#clean_song_split['song']
#print(audio_features.shape)
#print(len(embedding_songs_list))
#dfEmbdList = pd.DataFrame(embedding_songs_list)
#feather.write_dataframe(dfEmbdList, '/Users/dmarcous/git/wavenetmusicrecommendation/data/embedding_songs_list.feather')
#with open('/Users/dmarcous/git/wavenetmusicrecommendation/data/embedding_songs_list.pkl', mode='wb') as f:
#    pickle.dump(embedding_songs_list, f)

72721


In [12]:
## Train / Validation / Test - Split
#embedding_songs_list
#print((embedding_songs_list[1:5]))

In [13]:
import numpy as np
import pickle
#import keras
#from keras.layers import Dense, Input, Dropout
#from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Embedding
#from keras.layers.core import Lambda
#from keras import backend as K
#from keras.models import Model

In [382]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import keras
from keras.layers import Dense, Input, Dropout
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Lambda
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model
from keras import regularizers
from keras.layers.advanced_activations import LeakyReLU, ELU

## Build a CNN to predict latent factors

In [180]:
batch_size=64

def generate_mel_samples_train():
    songBatch = []
    batchCounter = 0
    while 1:
        for i in range(42):
            dataPath = './data/processedAudioFeatures/audio_features_part'+str(i)+'.feather'
            audioSmallDF = feather.read_dataframe(dataPath)
            for j in range(len(audioSmallDF.columns)):
                curSong = audioSmallDF.columns[j]
                if(curSong in splitted_train_songs):
                    songBatch.append(curSong)
                    batchCounter += 1
                if batchCounter == batch_size:
                    X = audioSmallDF.loc[:,songBatch].transpose().values.reshape(-1,599,128)
                    first_latent_factors = latent_factors.loc[:,songBatch]
                    Y = first_latent_factors.transpose().values.reshape(-1,1,40)
                    #print (X, Y)
                    batchCounter = 0
                    songBatch = []
                    yield (X, Y)


def generate_mel_samples_valid():
    songBatch = []
    batchCounter = 0
    while 1:
        for i in range(42):
            dataPath = './data/processedAudioFeatures/audio_features_part'+str(i)+'.feather'
            audioSmallDF = feather.read_dataframe(dataPath)
            for j in range(len(audioSmallDF.columns)):
                curSong = audioSmallDF.columns[j]
                if(curSong in splitted_valid_songs):
                    songBatch.append(curSong)
                    batchCounter += 1
                if batchCounter == batch_size:
                    X = audioSmallDF.loc[:,songBatch].transpose().values.reshape(-1,599,128)
                    first_latent_factors = latent_factors.loc[:,songBatch]
                    Y = first_latent_factors.transpose().values.reshape(-1,1,40)
                    #print (X, Y)
                    batchCounter = 0
                    songBatch = []
                    yield (X, Y)


In [187]:
inputs = Input(shape=(599,128))
conv1 = Conv1D(256, 4, strides=1, padding='valid', input_shape=(599, 128), activation='relu')(inputs)
# output (596,256)
mp1 = MaxPooling1D(pool_size=4)(conv1)
# output (100,149,256)

conv2 = Conv1D(256, 4, strides=1, padding='valid', activation='relu')(mp1)
# output (146,256)
mp2 = MaxPooling1D(pool_size=2)(conv2)
# output (73,256)

conv3 = Conv1D(512, 4, strides=1, padding='valid', activation='relu')(mp2)
# output (70,512)
mp3 = MaxPooling1D(pool_size=2)(conv3)
# output (35,512)

conv4 = Conv1D(512, 4, strides=1, padding='valid', activation='relu')(mp3)
# output (32,512)

ap = AveragePooling1D(pool_size=32)(conv4)
mp = MaxPooling1D(pool_size=32)(conv4)
def kerasl2norm(x):
    return K.reshape(K.sqrt(K.sum(K.square(x),axis=1)),(-1,1,512)) 
l2 = Lambda(kerasl2norm)(conv4)
concat = keras.layers.concatenate([ap,mp, l2])
# output (1536,)

#dense0 = Dense(2048, activation='relu')(concat)
dense1 = Dense(2048, activation='tanh')(concat)
dense1 = Dropout(0.1)(dense1)
# output (2048,)
dense2 = Dense(1024, activation='tanh')(dense1)
#dense2 = Dropout(0.2)(dense2)
# output (2048,)
output = Dense(40)(dense2)
# output (40,)

#dense1 = Dense(1400, activation='tanh')(dense0)
#dense1 = Dropout(0.1)(dense1)
#dense2 = Dense(1000, activation='tanh')(dense1)
#dense3 = Dense(600, activation='tanh')(dense2)
#dense4 = Dense(200, activation='tanh')(dense3)
#output = Dense(40)(dense4)

model = Model(inputs=[inputs], outputs=[output])
model.compile(optimizer='adam', loss='mean_squared_error')

In [188]:
#model.summary()

In [189]:
#model.fit([X], [Y], epochs=100, batch_size=64, validation_split=0.25)
model.fit_generator(generate_mel_samples_train(),
                    steps_per_epoch=np.floor(len(splitted_train_songs)/batch_size),
                    epochs=5,
                    validation_data=generate_mel_samples_valid(),
                    validation_steps = np.floor(len(splitted_valid_songs)/batch_size))

Epoch 1/5
681/681 [==============================] - 1323s - loss: 0.5130 - val_loss: 0.5033
Epoch 2/5
681/681 [==============================] - 1341s - loss: 0.5082 - val_loss: 0.5030
Epoch 3/5
681/681 [==============================] - 1325s - loss: 0.5075 - val_loss: 0.5029
Epoch 4/5
681/681 [==============================] - 1318s - loss: 0.5071 - val_loss: 0.5027
Epoch 5/5
681/681 [==============================] - 1377s - loss: 0.5071 - val_loss: 0.5026


In [190]:
model.save("./data/model_mels_v2")

## Use WaveNet embeddings as input

In [208]:
first_wavenet_embeddings_pairs = embedding_songs_list #[0:2]
first_wavenet_embedding_ids = [song_id for (song_id, song_path) in first_wavenet_embeddings_pairs if song_id in train_songs]
first_wavenet_embeddings = [np.load(song_path) for (song_id, song_path) in first_wavenet_embeddings_pairs if song_id in train_songs]
X = np.array([embedding.flatten() for embedding in first_wavenet_embeddings])
print(X.shape)
first_latent_factors = latent_factors.loc[:,first_wavenet_embedding_ids]
Y = first_latent_factors.transpose().values.reshape(-1,40)
print(Y.shape)

(58176, 2000)
(58176, 40)


In [383]:
inputs = Input(shape=(2000,))

dense1 = Dense(1600, activation='relu',
               kernel_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               activity_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               bias_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002))(inputs)
dense1 = Dropout(0.055)(dense1)
dense1 = LeakyReLU(0.2)(dense1)
dense1 = BatchNormalization()(dense1)
dense2 = Dense(1300, activation='relu',
               kernel_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               activity_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               bias_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002))(dense1)
dense2 = Dropout(0.055)(dense2)
dense2 = LeakyReLU(0.2)(dense2)
#dense2 = BatchNormalization()(dense2)
dense3 = Dense(1000, activation='relu',
               kernel_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               activity_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               bias_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002))(dense2)
dense3 = Dropout(0.055)(dense3)
dense3 = LeakyReLU(0.2)(dense3)
#dense3 = BatchNormalization()(dense3)
dense4 = Dense(700, activation='relu',
               kernel_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               activity_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               bias_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002))(dense3)
dense4 = Dropout(0.055)(dense4)
dense4 = LeakyReLU(0.2)(dense4)
#dense4 = BatchNormalization()(dense4)
dense5 = Dense(500, activation='relu',
               kernel_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               activity_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               bias_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002))(dense4)
dense5 = Dropout(0.055)(dense5)
dense5 = LeakyReLU(0.2)(dense5)
#dense5 = BatchNormalization()(dense5)
dense6 = Dense(250, activation='relu',
               kernel_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               activity_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               bias_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002))(dense5)
dense6 = Dropout(0.055)(dense6)
dense6 = LeakyReLU(0.2)(dense6)
#dense6 = BatchNormalization()(dense6)
dense7 = Dense(100, activation='relu',
               kernel_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               activity_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002),
               bias_regularizer=regularizers.l1_l2(l1=0.00000002,l2=0.00000002))(dense6)
dense7 = Dropout(0.055)(dense7)
#dense7 = BatchNormalization()(dense7)
dense7 = LeakyReLU(0.2)(dense7)
output = Dense(40)(dense7)

model = Model(inputs=[inputs], outputs=[output])
opt = Adam(lr=1e-4)
model.compile(optimizer=opt, loss='mean_squared_error')

In [384]:
#model.summary()
#print(X.shape)

In [ ]:
model.fit([X], [Y], epochs=10, batch_size=64, validation_split=0.25)

Train on 43632 samples, validate on 14544 samples
Epoch 1/10
43632/43632 [==============================] - 179s - loss: 0.4976 - val_loss: 0.5237
Epoch 2/10
43632/43632 [==============================] - 185s - loss: 0.4973 - val_loss: 0.5262
Epoch 3/10
43632/43632 [==============================] - 196s - loss: 0.4974 - val_loss: 0.5187
Epoch 4/10
43632/43632 [==============================] - 197s - loss: 0.4969 - val_loss: 0.5210
Epoch 5/10
43632/43632 [==============================] - 192s - loss: 0.4972 - val_loss: 0.5199
Epoch 6/10
43632/43632 [==============================] - 202s - loss: 0.4967 - val_loss: 0.5196
Epoch 7/10
43632/43632 [==============================] - 219s - loss: 0.4964 - val_loss: 0.5238
Epoch 8/10
43632/43632 [==============================] - 223s - loss: 0.4963 - val_loss: 0.5183
Epoch 9/10
43632/43632 [==============================] - 232s - loss: 0.4975 - val_loss: 0.5178
Epoch 10/10
43632/43632 [==============================] - 210s - loss: 0.496

In [359]:
model.save("./data/model_wavenet_v7")

## Hybrid WaveNet+Mel 

In [16]:
inputs = Input(shape=(599,128))
conv1 = Conv1D(256, 4, strides=1, padding='valid', input_shape=(599, 128), activation='relu')(inputs)
# output (596,256)
mp1 = MaxPooling1D(pool_size=4)(conv1)
# output (100,149,256)

conv2 = Conv1D(256, 4, strides=1, padding='valid', activation='relu')(mp1)
# output (146,256)
mp2 = MaxPooling1D(pool_size=2)(conv2)
# output (73,256)

conv3 = Conv1D(512, 4, strides=1, padding='valid', activation='relu')(mp2)
# output (70,512)
mp3 = MaxPooling1D(pool_size=2)(conv3)
# output (35,512)

conv4 = Conv1D(512, 4, strides=1, padding='valid', activation='relu')(mp3)
# output (32,512)

ap = AveragePooling1D(pool_size=32)(conv4)
mp = MaxPooling1D(pool_size=32)(conv4)
def kerasl2norm(x):
    return K.reshape(K.sqrt(K.sum(K.square(x),axis=1)),(-1,1,512)) 
l2 = Lambda(kerasl2norm)(conv4)

WAVENETINPUT = ?????? Input(shape=(2000,))

concat = keras.layers.concatenate([ap,mp, l2, WAVENETINPUT])
# output (1536,)

dense1 = Dense(4048, activation='relu')(concat)
dense1 = Dropout(0.25)(dense1)
# output (2048,)
dense2 = Dense(2048, activation='relu')(dense1)
dense2 = Dropout(0.25)(dense2)
# output (2048,)
output = Dense(40)(dense2)
# output (40,)

model = Model(inputs=[inputs], outputs=[output])
model.compile(optimizer='adam', loss='mean_squared_error')

In [50]:
model.summary()

(58176, 2000)


In [46]:
#model.fit([X], [Y], epochs=100, batch_size=64, validation_split=0.25)
#model.fit_generator(generate_mel_samples_train(),
#                    steps_per_epoch=np.floor(len(splitted_train_songs)/batch_size),
 #                   epochs=5,
 #                   validation_data=generate_mel_samples_valid(),
 #                   validation_steps = np.floor(len(splitted_valid_songs)/batch_size))

## Use WaveNet weights to initialize

In [19]:
#weights_first_wavenet_embeddings_pairs = embedding_songs_list 
#weights_first_wavenet_embedding_ids = [song_id for (song_id, song_path) in weights_first_wavenet_embeddings_pairs if song_id in train_songs]
#weights_first_wavenet_embeddings = {song_id: np.load(song_path).flatten() for (song_id, song_path) in weights_first_wavenet_embeddings_pairs if song_id in train_songs}
#weights_first_latent_factors = latent_factors.loc[:,weights_first_wavenet_embedding_ids]
#weights_label_latent_factors = weights_first_latent_factors

In [20]:
#weights_wavenet_embeddings = weights_first_wavenet_embeddings

In [21]:
# Tokenize songs
songs_dict = {s:i for (i,s) in enumerate(weights_wavenet_embeddings.keys())}
songs_dict_reversed = {i:s for (s,i) in songs_dict.items()}

In [22]:
# Create initialization for the embedding layer
def my_init(shape = None, dtype = None):
    mat = []
    for i in range(len(songs_dict_reversed)):
        mat.append(weights_wavenet_embeddings[songs_dict_reversed[i]])
    mat = np.array(mat)
    return K.variable(mat)

In [23]:
song_input = Input(shape = (1,))
embedding = Embedding(input_dim=len(songs_dict), output_dim=2000,
                      embeddings_initializer=my_init, input_length=1)(song_input)#,shape=((2000,)))

dense1 = Dense(1400, activation='relu')(embedding)
dense1 = Dropout(0.25)(dense1)
# output (2048,)
dense2 = Dense(600, activation='relu')(dense1)
dense2 = Dropout(0.25)(dense2)
# output (2048,)
output = Dense(40)(dense2)
# output (40,)

model = Model(inputs=[song_input], outputs=[output])
model.compile(optimizer='adam', loss='mean_squared_error')

In [45]:
#model.summary()

In [25]:
X = np.array(range(len(weights_wavenet_embeddings)))
Y = np.array([weights_label_latent_factors.loc[:,songs_dict_reversed[i]] for i in X]).reshape(-1,1,40)
print(X.shape)
print(Y.shape)

(58176,)
(58176, 1, 40)


In [44]:
#model.fit([X], [Y], epochs=5, batch_size=64, validation_split=0.25)

## Evaluation

In [191]:
def generate_mel_samples_predict():
    songBatch = []
    batchCounter = 0
    while 1:
        for i in range(42):
            dataPath = './data/processedAudioFeatures/audio_features_part'+str(i)+'.feather'
            audioSmallDF = feather.read_dataframe(dataPath)
            for j in range(len(audioSmallDF.columns)):
                curSong = audioSmallDF.columns[j]
                if(curSong in train_songs or curSong in test_songs):
                    songBatch.append(curSong)
                    batchCounter += 1
                if batchCounter == 11:
                    X = audioSmallDF.loc[:,songBatch].transpose().values.reshape(-1,599,128)
                    first_latent_factors = latent_factors.loc[:,songBatch]
                    batchCounter = 0
                    songBatch = []
                    yield (X)

In [192]:
# Net 1 
test_predictions = model.predict_generator(generate_mel_samples_predict(),
                                           #steps = 1)
                                           steps = np.floor((len(test_songs)+len(train_songs))/11))
df_test_predictions = pd.DataFrame(test_predictions.reshape(-1,40).transpose(),columns=latent_factors.columns)
print(df_test_predictions.shape)
latent_predictions = scaler.inverse_transform(df_test_predictions) 
df_latent_predictions = pd.DataFrame(latent_predictions,columns=latent_factors.columns)
latent_test_predictions = df_latent_predictions.loc[:,test_songs]
feather.write_dataframe(latent_test_predictions,"./data/mel_predictions_v2.feather")

(40, 72721)


In [360]:
# Net 2 
first_wavenet_embeddings_pairs = embedding_songs_list #[0:2]
first_wavenet_embedding_ids = [song_id for (song_id, song_path) in first_wavenet_embeddings_pairs]
first_wavenet_embeddings = [np.load(song_path) for (song_id, song_path) in first_wavenet_embeddings_pairs]
pred_X = np.array([embedding.flatten() for embedding in first_wavenet_embeddings])
test_predictions = model.predict(pred_X, batch_size = 64)

df_test_predictions = pd.DataFrame(test_predictions.transpose(),columns=latent_factors.columns)
print(df_test_predictions.shape)
latent_predictions = scaler.inverse_transform(df_test_predictions) 
df_latent_predictions = pd.DataFrame(latent_predictions,columns=latent_factors.columns)
latent_test_predictions = df_latent_predictions.loc[:,test_songs]
feather.write_dataframe(latent_test_predictions,"./data/wavenet_predictions_v7.feather")

(40, 72721)
